# OmniGenome - A Demonstration based on RNA Secondary Structure Prediction
GitHub: https://github.com/yangheng95/OmniGenome
OmniGenome Hub: Huggingface Spaces

## Introduction
OmniGenome is a comprehensive package designed for pretrained genomic foundation models (FMs) development and FM benchmark. 
OmniGenome have the following key features:
- Automated genomic FM benchmarking on public genomic datasets
- Scalable genomic FM training and fine-tuning on genomic tasks
- Diversified genomic FMs implementation
- Easy-to-use pipeline for genomic FM development with no coding expertise required
- Accessible OmniGenome Hub for sharing FMs, datasets, and pipelines
- Extensive documentation and tutorials for genomic FM development

We begin to introduce OmniGenome by delivering a demonstration to train a model to predict RNA secondary structures. The dataset used in this demonstration is the bpRNA dataset which contains RNA sequences and their corresponding secondary structures. The secondary structure of an RNA sequence is a set of base pairs that describe the folding of the RNA molecule. The secondary structure of an RNA sequence is important for understanding the function of the RNA molecule. In this demonstration, we will train a model to predict the secondary structure of an RNA sequence given its primary sequence.

## Requirements
OmniGenome requires the following recommended dependencies:
- Python 3.9+
- PyTorch 2.0.0+
- Transformers 4.37.0+
- Pandas 1.3.3+
- Others in case of specific tasks

pip install OmniGenome


## Fine-tuning Genomic FMs for RNA Secondary Structure Prediction

### Step 1: Import Libraries

In [1]:
import autocuda
import torch
from metric_visualizer import MetricVisualizer

from omnigenome import OmniGenomeDatasetForTokenClassification
from omnigenome import ClassificationMetric
from omnigenome import OmniSingleNucleotideTokenizer, OmniKmersTokenizer
from omnigenome import OmniGenomeModelForTokenClassification
from omnigenome import Trainer

C:\Users\hengu\miniconda3\envs\py312\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\hengu\miniconda3\envs\py312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



                                
   **  +----------- **           ___                     _ 
  @@                 @@         / _ \  _ __ ___   _ __  (_)
 @@* #============== *@@       | | | || '_ ` _ \ | '_ \ | |
 @@*                 *@@       | |_| || | | | | || | | || |
 *@@  +------------ *@@         \___/ |_| |_| |_||_| |_||_|
  *@*               @@*       
   *@@  #========= @@*        
    *@@*         *@@*          
      *@@  +---@@@*              ____  
        *@@*   **               / ___|  ___  _ __    ___   _ __ ___    ___ 
          **@**                | |  _  / _ \| '_ \  / _ \ | '_ ` _ \  / _ \ 
        *@@* *@@*              | |_| ||  __/| | | || (_) || | | | | ||  __/ 
      *@@ ---+  @@*             \____| \___||_| |_| \___/ |_| |_| |_| \___| 
    *@@*         *@@*          
   *@@ =========#  @@*         
  *@@               @@*        
 *@@ -------------+  @@*        ____                      _   
 @@                   @@       | __ )   ___  _ __    ___ | |__ 
 @

### Step 2: Define and Initialize the Tokenizer

In [2]:
# Predefined dataset label mapping
label2id = {"(": 0, ")": 1, ".": 2}  # label2id is a dictionary that maps the labels to indices for prediction.

# The is FM is exclusively powered by the OmniGenome package
model_name_or_path = "anonymous8/OmniGenome-186M"

# Generally, we use the tokenizers from transformers library, such as AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# However, OmniGenome provides specialized tokenizers for genomic data, such as single nucleotide tokenizer and k-mers tokenizer
# we can force the tokenizer to be used in the model
tokenizer = OmniSingleNucleotideTokenizer.from_pretrained(model_name_or_path)

### Step 3: Define and Initialize the Model

In [3]:
# We have implemented a diverse set of genomic models in OmniGenome, please refer to the documentation for more details
ssp_model = OmniGenomeModelForTokenClassification(
    model_name_or_path,
    tokenizer=tokenizer,
    label2id=label2id,
)

A new version of the following files was downloaded from https://huggingface.co/anonymous8/OmniGenome-186M:
- configuration_omnigenome.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/anonymous8/OmniGenome-186M:
- modeling_omnigenome.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Some weights of OmniGenomeModel were not initialized from the model checkpoint at anonymous8/OmniGenome-186M and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[2025-06-28 20:05:38] [OmniGenome 0.2.7alpha]  Model Name: OmniGenomeModelForTokenClassification
Model Metadata: {'library_name': 'OmniGenome', 'omnigenome_version': '0.2.7alpha', 'torch_version': '2.7.0+cu128+cu12.8+git134179474539648ba7dee1317959529fbd0e7f89', 'transformers_version': '4.52.4', 'model_cls': 'OmniGenomeModelForTokenClassification', 'tokenizer_cls': 'OmniSingleNucleotideTokenizer', 'model_name': 'OmniGenomeModelForTokenClassification'}
Base Model Name: anonymous8/OmniGenome-186M
Model Type: omnigenome
Model Architecture: None
Model Parameters: 185.886801 M
Model Config: OmniGenomeConfig {
  "OmniGenomefold_config": null,
  "attention_probs_dropout_prob": 0.0,
  "auto_map": {
    "AutoConfig": "anonymous8/OmniGenome-186M--configuration_omnigenome.OmniGenomeConfig",
    "AutoModel": "anonymous8/OmniGenome-186M--modeling_omnigenome.OmniGenomeModel",
    "AutoModelForMaskedLM": "anonymous8/OmniGenome-186M--modeling_omnigenome.OmniGenomeForMaskedLM",
    "AutoModelForSeq2Seq

### Step 4: Define and Load the Dataset

In [4]:
# necessary hyperparameters
epochs = 10
learning_rate = 2e-5
weight_decay = 1e-5
batch_size = 8
max_length = 512
seeds = [45]  # Each seed will be used for one run


# Load the dataset according to the path
train_file = "toy_datasets/Archive2/train.json"
test_file = "toy_datasets/Archive2/test.json"
valid_file = "toy_datasets/Archive2/valid.json"

train_set = OmniGenomeDatasetForTokenClassification(
    data_source=train_file,
    tokenizer=tokenizer,
    label2id=label2id,
    max_length=max_length,
)
test_set = OmniGenomeDatasetForTokenClassification(
    data_source=test_file,
    tokenizer=tokenizer,
    label2id=label2id,
    max_length=max_length,
)
valid_set = OmniGenomeDatasetForTokenClassification(
    data_source=valid_file,
    tokenizer=tokenizer,
    label2id=label2id,
    max_length=max_length,
)
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True
)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

[2025-06-28 20:05:38] [OmniGenome 0.2.7alpha]  Detected max_length=512 in the dataset, using it as the max_length.
[2025-06-28 20:05:38] [OmniGenome 0.2.7alpha]  Loading data from toy_datasets/Archive2/train.json...
[2025-06-28 20:05:38] [OmniGenome 0.2.7alpha]  Loaded 608 examples from toy_datasets/Archive2/train.json
[2025-06-28 20:05:38] [OmniGenome 0.2.7alpha]  Detected shuffle=True, shuffling the examples...


100%|██████████| 608/608 [00:00<00:00, 2052.78it/s]


[2025-06-28 20:05:38] [OmniGenome 0.2.7alpha]  Max sequence length updated -> Reset max_length=504, label_padding_length=504
[2025-06-28 20:05:38] [OmniGenome 0.2.7alpha]  {'avg_seq_len': np.float64(130.54276315789474), 'max_seq_len': np.int64(501), 'min_seq_len': np.int64(56), 'avg_label_len': np.float64(504.0), 'max_label_len': np.int64(504), 'min_label_len': np.int64(504)}
[2025-06-28 20:05:38] [OmniGenome 0.2.7alpha]  Preview of the first two samples in the dataset:
[2025-06-28 20:05:38] [OmniGenome 0.2.7alpha]  {'input_ids': tensor([0, 6, 5, 5, 6, 6, 6, 5, 6, 9, 9, 6, 9, 6, 6, 5, 6, 4, 6, 5, 6, 5, 9, 9,
        6, 9, 4, 4, 9, 5, 5, 4, 4, 6, 9, 6, 4, 5, 9, 6, 6, 6, 4, 6, 6, 5, 4, 9,
        6, 9, 6, 6, 9, 6, 6, 5, 6, 6, 4, 9, 6, 6, 4, 6, 9, 6, 4, 6, 4, 6, 6, 4,
        6, 6, 4, 6, 9, 5, 5, 9, 6, 6, 5, 9, 6, 4, 9, 6, 5, 9, 9, 6, 6, 5, 6, 6,
        9, 6, 6, 5, 6, 4, 5, 5, 6, 4, 6, 5, 6, 9, 5, 5, 4, 6, 6, 5, 5, 4, 4, 6,
        5, 6, 9, 6, 6, 5, 6, 9, 5, 6, 4, 5, 4, 4, 6, 6, 9, 6, 9,

100%|██████████| 82/82 [00:00<00:00, 1950.21it/s]


[2025-06-28 20:05:38] [OmniGenome 0.2.7alpha]  Max sequence length updated -> Reset max_length=328, label_padding_length=328
[2025-06-28 20:05:38] [OmniGenome 0.2.7alpha]  {'avg_seq_len': np.float64(131.23170731707316), 'max_seq_len': np.int64(321), 'min_seq_len': np.int64(67), 'avg_label_len': np.float64(328.0), 'max_label_len': np.int64(328), 'min_label_len': np.int64(328)}
[2025-06-28 20:05:38] [OmniGenome 0.2.7alpha]  Preview of the first two samples in the dataset:
[2025-06-28 20:05:38] [OmniGenome 0.2.7alpha]  {'input_ids': tensor([0, 6, 6, 4, 6, 4, 6, 5, 9, 6, 9, 5, 5, 6, 4, 6, 9, 6, 6, 9, 5, 6, 4, 4,
        6, 6, 4, 6, 5, 4, 5, 6, 4, 9, 9, 6, 6, 4, 4, 4, 9, 5, 6, 9, 6, 9, 4, 6,
        6, 5, 6, 6, 9, 5, 4, 4, 5, 9, 5, 5, 6, 9, 5, 9, 5, 4, 4, 6, 6, 6, 9, 9,
        5, 6, 4, 4, 9, 5, 5, 5, 9, 9, 6, 5, 9, 5, 9, 5, 5, 6, 5, 5, 4, 2, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

100%|██████████| 76/76 [00:00<00:00, 2034.90it/s]


[2025-06-28 20:05:38] [OmniGenome 0.2.7alpha]  Max sequence length updated -> Reset max_length=312, label_padding_length=312
[2025-06-28 20:05:38] [OmniGenome 0.2.7alpha]  {'avg_seq_len': np.float64(117.39473684210526), 'max_seq_len': np.int64(308), 'min_seq_len': np.int64(60), 'avg_label_len': np.float64(312.0), 'max_label_len': np.int64(312), 'min_label_len': np.int64(312)}
[2025-06-28 20:05:38] [OmniGenome 0.2.7alpha]  Preview of the first two samples in the dataset:
[2025-06-28 20:05:38] [OmniGenome 0.2.7alpha]  {'input_ids': tensor([0, 6, 6, 4, 5, 5, 9, 9, 9, 4, 6, 5, 9, 5, 4, 6, 9, 9, 6, 6, 9, 9, 4, 6,
        4, 6, 5, 4, 9, 5, 5, 6, 6, 5, 9, 5, 4, 9, 4, 4, 5, 5, 6, 6, 4, 5, 6, 6,
        9, 5, 4, 9, 9, 6, 6, 9, 9, 5, 4, 4, 6, 9, 5, 5, 4, 4, 9, 4, 4, 6, 6, 9,
        5, 5, 4, 5, 5, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

### Step 5: Define the Metrics
We have implemented a diverse set of genomic metrics in OmniGenome, please refer to the documentation for more details.
Users can also define their own metrics by inheriting the `OmniGenomeMetric` class. 
The `compute_metrics` can be a metric function list and each metric function should return a dictionary of metrics.

In [5]:
compute_metrics = [
    ClassificationMetric(ignore_y=-100).accuracy_score,
    ClassificationMetric(ignore_y=-100, average="macro").f1_score,
    ClassificationMetric(ignore_y=-100).matthews_corrcoef,
]


## Step 6: Define and Initialize the Trainer

In [6]:
# Initialize the MetricVisualizer for logging the metrics
mv = MetricVisualizer(name="OmniGenome-186M-SSP")

for seed in seeds:
    optimizer = torch.optim.AdamW(
        ssp_model.parameters(), lr=learning_rate, weight_decay=weight_decay
    )
    trainer = Trainer(
        model=ssp_model,
        train_loader=train_loader,
        eval_loader=valid_loader,
        test_loader=test_loader,
        batch_size=batch_size,
        epochs=epochs,
        optimizer=optimizer,
        compute_metrics=compute_metrics,
        seeds=seed,
        device=autocuda.auto_cuda(),
    )

    metrics = trainer.train()
    test_metrics = metrics["test"][-1]
    mv.log(model_name_or_path.split("/")[-1], "F1", test_metrics["f1_score"])
    mv.log(
        model_name_or_path.split("/")[-1],
        "Accuracy",
        test_metrics["accuracy_score"],
    )
    print(metrics)
    mv.summary()

D:\OneDrive - University of Exeter\AIProjects\OmniGenomeBench\omnigenome\src\trainer\trainer.py:140: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = GradScaler()
Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]D:\OneDrive - University of Exeter\AIProjects\OmniGenomeBench\omnigenome\src\abc\abstract_model.py:169: UserWarning: Warning: Ignored keys in inputs: {'x'}
  warnings.warn(f"Warning: Ignored keys in inputs: {ignored_keys}")
Evaluating: 100%|██████████| 10/10 [00:00<00:00, 10.47it/s]


[2025-06-28 20:05:40] [OmniGenome 0.2.7alpha]  {'accuracy_score': 0.3578107183580388, 'f1_score': 0.3218228611028748, 'matthews_corrcoef': 0.10040023403620794}


Epoch 1/10 Loss:   0%|          | 0/76 [00:00<?, ?it/s]D:\OneDrive - University of Exeter\AIProjects\OmniGenomeBench\omnigenome\src\abc\abstract_model.py:169: UserWarning: Warning: Ignored keys in inputs: {'x'}
  warnings.warn(f"Warning: Ignored keys in inputs: {ignored_keys}")
Evaluating: 100%|██████████| 10/10 [00:00<00:00, 16.15it/s]


[2025-06-28 20:05:56] [OmniGenome 0.2.7alpha]  {'accuracy_score': 0.8904218928164196, 'f1_score': 0.8925439071092223, 'matthews_corrcoef': 0.8338644169965754}


Epoch 2/10 Loss:   0%|          | 0/76 [00:00<?, ?it/s]D:\OneDrive - University of Exeter\AIProjects\OmniGenomeBench\omnigenome\src\abc\abstract_model.py:169: UserWarning: Warning: Ignored keys in inputs: {'x'}
  warnings.warn(f"Warning: Ignored keys in inputs: {ignored_keys}")
Evaluating: 100%|██████████| 10/10 [00:00<00:00, 16.30it/s]


...

[2025-06-28 20:08:04] [OmniGenome 0.2.7alpha]  {'accuracy_score': 0.9201824401368301, 'f1_score': 0.9216819609151036, 'matthews_corrcoef': 0.87857191405743}


Epoch 10/10 Loss:   0%|          | 0/76 [00:00<?, ?it/s]D:\OneDrive - University of Exeter\AIProjects\OmniGenomeBench\omnigenome\src\abc\abstract_model.py:169: UserWarning: Warning: Ignored keys in inputs: {'x'}
  warnings.warn(f"Warning: Ignored keys in inputs: {ignored_keys}")
Evaluating: 100%|██████████| 10/10 [00:00<00:00, 15.82it/s]


[2025-06-28 20:08:25] [OmniGenome 0.2.7alpha]  {'accuracy_score': 0.9190421892816419, 'f1_score': 0.9207599078115599, 'matthews_corrcoef': 0.8753664046414782}


Testing:   0%|          | 0/11 [00:00<?, ?it/s]D:\OneDrive - University of Exeter\AIProjects\OmniGenomeBench\omnigenome\src\abc\abstract_model.py:169: UserWarning: Warning: Ignored keys in inputs: {'x'}
  warnings.warn(f"Warning: Ignored keys in inputs: {ignored_keys}")
Testing: 100%|██████████| 11/11 [00:00<00:00, 16.13it/s]


[2025-06-28 20:08:26] [OmniGenome 0.2.7alpha]  {'accuracy_score': 0.9093139567802209, 'f1_score': 0.9111728275698807, 'matthews_corrcoef': 0.862345618945574}
{'valid': [{'accuracy_score': 0.3578107183580388, 'f1_score': 0.3218228611028748, 'matthews_corrcoef': 0.10040023403620794}, {'accuracy_score': 0.8904218928164196, 'f1_score': 0.8925439071092223, 'matthews_corrcoef': 0.8338644169965754}, {'accuracy_score': 0.9094640820980616, 'f1_score': 0.9114027245736306, 'matthews_corrcoef': 0.8612940824481876}, {'accuracy_score': 0.9173318129988598, 'f1_score': 0.9191446314511356, 'matthews_corrcoef': 0.8744695320581789}, {'accuracy_score': 0.9135689851767389, 'f1_score': 0.915234317802641, 'matthews_corrcoef': 0.8672732258383623}, {'accuracy_score': 0.9182440136830102, 'f1_score': 0.9200067747454056, 'matthews_corrcoef': 0.8746403283470774}, {'accuracy_score': 0.9185860889395667, 'f1_score': 0.9204274150890576, 'matthews_corrcoef': 0.8746835983620637}, {'accuracy_score': 0.9194982896237173, '

### Step 7. Experimental Results Visualization
The experimental results are visualized in the following plots. The plots show the F1 score and accuracy of the model on the test set for each run. The average F1 score and accuracy are also shown.

### Step 8. Model Checkpoint for Sharing
The model checkpoint can be saved and shared with others for further use. The model checkpoint can be loaded using the following code:

**Regular checkpointing and resuming are good practices to save the model at different stages of training.**

In [7]:
path_to_save = "OmniGenome-186M-SSP"
ssp_model.save(path_to_save, overwrite=True)

# Load the model checkpoint
ssp_model = ssp_model.load(path_to_save)
results = ssp_model.inference("CAGUGCCGAGGCCACGCGGAGAACGAUCGAGGGUACAGCACUA")
print(results["predictions"])
print("logits:", results["logits"])

[2025-06-28 20:08:30] [OmniGenome 0.2.7alpha]  The model is saved to OmniGenome-186M-SSP.
[2025-06-28 20:08:30] [OmniGenome 0.2.7alpha]  Warning: The value of the key torch_dtype in the loaded model is torch.float16, but the current value is float16.
[2025-06-28 20:08:30] [OmniGenome 0.2.7alpha]  Warning: The value of the key _name_or_path in the loaded model is OmniGenome-186M-SSP, but the current value is anonymous8/OmniGenome-186M.
[2025-06-28 20:08:30] [OmniGenome 0.2.7alpha]  Warning: The value of the key _commit_hash in the loaded model is None, but the current value is c48fefaf3431af322aff7fbe9a64387ee189a0de.
[2025-06-28 20:08:30] [OmniGenome 0.2.7alpha]  Warning: The value of the key transformers_version in the loaded model is 4.52.4, but the current value is 4.41.0.dev0.
[2025-06-28 20:08:30] [OmniGenome 0.2.7alpha]  Restored loss function: CrossEntropyLoss from torch.nn.modules.loss


C:\Users\hengu\miniconda3\envs\py312\Lib\site-packages\transformers\tokenization_utils_base.py:2714: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
D:\OneDrive - University of Exeter\AIProjects\OmniGenomeBench\omnigenome\src\abc\abstract_model.py:169: UserWarning: Warning: Ignored keys in inputs: {'x'}
  warnings.warn(f"Warning: Ignored keys in inputs: {ignored_keys}")


['.', '(', '(', '(', '(', '(', '.', '.', '.', '.', '(', '(', '(', '.', '(', '.', '(', '(', '.', '.', '.', '.', '.', ')', ')', '.', '.', ')', ')', '.', ')', ')', ')', ')', '.', '.', '.', ')', ')', ')', ')', ')', '.']
logits: tensor([[4.7709e-03, 4.3566e-03, 9.9087e-01],
        [2.1867e-04, 1.1555e-04, 9.9967e-01],
        [9.9962e-01, 1.6863e-04, 2.1297e-04],
        [9.9976e-01, 1.5027e-04, 9.1302e-05],
        [9.9975e-01, 1.4544e-04, 1.0465e-04],
        [9.9981e-01, 8.7038e-05, 1.0526e-04],
        [9.9975e-01, 1.3791e-04, 1.1497e-04],
        [2.5388e-04, 1.0671e-04, 9.9964e-01],
        [1.8994e-04, 7.0000e-05, 9.9974e-01],
        [1.4346e-04, 8.8145e-05, 9.9977e-01],
        [2.8687e-04, 5.7227e-05, 9.9966e-01],
        [9.9942e-01, 3.1429e-04, 2.6404e-04],
        [9.9962e-01, 2.2909e-04, 1.5214e-04],
        [9.9951e-01, 3.2460e-04, 1.6885e-04],
        [2.1617e-04, 7.6360e-05, 9.9971e-01],
        [9.9946e-01, 3.4337e-04, 1.9187e-04],
        [1.1877e-01, 1.3877e-03, 8.7984e


# What if someone doesn't know how to initialize the model?

In [8]:
# We can load the model checkpoint using the ModelHub
from omnigenome import ModelHub

ssp_model = ModelHub.load("OmniGenome-186M-SSP")
results = ssp_model.inference("CAGUGCCGAGGCCACGCGGAGAACGAUCGAGGGUACAGCACUA")
print(results["predictions"])
print("logits:", results["logits"])

[2025-06-28 20:08:37] [OmniGenome 0.2.7alpha]  Model Name: OmniGenomeModelForTokenClassification
Model Metadata: {'library_name': 'OmniGenome', 'omnigenome_version': '0.2.7alpha', 'torch_version': '2.7.0+cu128+cu12.8+git134179474539648ba7dee1317959529fbd0e7f89', 'transformers_version': '4.52.4', 'model_cls': 'OmniGenomeModelForTokenClassification', 'tokenizer_cls': 'OmniSingleNucleotideTokenizer', 'model_name': 'OmniGenomeModelForTokenClassification'}
Base Model Name: OmniGenome-186M-SSP
Model Type: omnigenome
Model Architecture: ['OmniGenomeModel']
Model Parameters: 185.886801 M
Model Config: OmniGenomeConfig {
  "OmniGenomefold_config": null,
  "architectures": [
    "OmniGenomeModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "auto_map": {
    "AutoConfig": "anonymous8/OmniGenome-186M--configuration_omnigenome.OmniGenomeConfig",
    "AutoModel": "anonymous8/OmniGenome-186M--modeling_omnigenome.OmniGenomeModel",
    "AutoModelForMaskedLM": "anonymous8/OmniGenome-186M--modeling_omn

## Step 8. Model Inference

In [9]:
examples = [
    "GCUGGGAUGUUGGCUUAGAAGCAGCCAUCAUUUAAAGAGUGCGUAACAGCUCACCAGC",
    "AUCUGUACUAGUUAGCUAACUAGAUCUGUAUCUGGCGGUUCCGUGGAAGAACUGACGUGUUCAUAUUCCCGACCGCAGCCCUGGGAGACGUCUCAGAGGC",
]

results = ssp_model.inference(examples)
structures = ["".join(prediction) for prediction in results["predictions"]]
print(results)
print(structures)

{'predictions': [['(', '(', '(', '(', '(', '(', '(', '(', '(', '.', '(', '(', '(', '(', '(', '.', '.', '.', '.', '.', '.', '.', ')', ')', ')', ')', ')', '.', ')', ')', ')', ')', '.', '.', '.', '.', '(', '(', '(', '(', '.', '.', '.', '.', '.', '.', '.', '.', ')', ')', ')', ')', '.', ')', ')', ')', ')', ')'], ['.', '.', '.', '.', '.', '.', '.', '(', '(', '(', '(', '(', '.', '.', '.', '.', '.', '.', ')', ')', ')', ')', ')', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '(', '(', '(', '(', '(', '.', '.', '.', '.', '.', '(', '(', '(', '.', '(', '(', '(', '(', '.', '.', '.', '.', '.', '.', ')', ')', ')', ')', '.', '.', '.', ')', ')', ')', ')', '.', ')', ')', ')', ')', ')', ')', '.', '(', '(', '(', '(', '(', '(', '(', '(', '(', '.', '.', '.', '.', ')', ')', ')', ')', ')', ')', '.', ')', ')', ')']], 'logits': tensor([[[7.2050e-04, 6.2609e-04, 9.9854e-01],
         [9.9951e-01, 2.0778e-04, 1.2434e-04],
         [9.9951e-01, 1.7190e-04, 1.3137e-04],
         [9.9951e-01, 2.0146e-04, 1.5

### Step 9. Pipeline Creation
The OmniGenome package provides pipelines for genomic FM development. The pipeline can be used to train, fine-tune, and evaluate genomic FMs. The pipeline can be used with a single command to train a genomic FM on a dataset. The pipeline can also be used to fine-tune a pre-trained genomic FM on a new dataset. The pipeline can be used to evaluate the performance of a genomic FM on a dataset. The pipeline can be used to generate predictions using a genomic FM.

In [10]:
# from omnigenome import Pipeline, PipelineHub
# 
# pipeline = Pipeline(
#     name="OmniGenome-186M-SSP-Pipeline",
#     # model_name_or_path="OmniGenome-186M-SSP",  # The model name or path can be specified
#     # tokenizer="OmniGenome-186M-SSP",  # The tokenizer can be specified
#     model_name_or_path=ssp_model,
#     tokenizer=ssp_model.tokenizer,
#     datasets={
#         "train": "toy_datasets/train.json",
#         "test": "toy_datasets/test.json",
#         "valid": "toy_datasets/valid.json",
#     },
#     trainer=trainer,
#     device=ssp_model.model.device,
# )

### Using the Pipeline

In [11]:
# results = pipeline(examples[0])
# print(results)
# 
# pipeline.train()
# 
# pipeline.save("OmniGenome-186M-SSP-Pipeline", overwrite=True)
# 
# pipeline = PipelineHub.load("OmniGenome-186M-SSP-Pipeline")
# results = pipeline(examples)
# print(results)

## Web Demo for RNA Secondary Structure Prediction

In [12]:
import os
import time
import base64
import tempfile
from pathlib import Path
import json
import numpy as np
import gradio as gr
import RNA
from omnigenome import ModelHub

# 加载模型
ssp_model = ModelHub.load("OmniGenome-186M-SSP")

# 临时 SVG 存储目录
TEMP_DIR = Path(tempfile.mkdtemp())
print(f"Using temporary directory: {TEMP_DIR}")


def ss_validity_loss(rna_strct: str) -> float:
    left = right = 0
    dots = rna_strct.count('.')
    for c in rna_strct:
        if c == '(':
            left += 1
        elif c == ')':
            if left:
                left -= 1
            else:
                right += 1
        elif c != '.':
            raise ValueError(f"Invalid char {c}")
    return (left + right) / (len(rna_strct) - dots + 1e-8)


def find_invalid_positions(struct: str) -> list:
    stack, invalid = [], []
    for i, c in enumerate(struct):
        if c == '(': stack.append(i)
        elif c == ')':
            if stack:
                stack.pop()
            else:
                invalid.append(i)
    invalid.extend(stack)
    return invalid


def generate_svg_datauri(rna_seq: str, struct: str) -> str:
    """生成 SVG 并返回 Base64 URI"""
    try:
        path = TEMP_DIR / f"{hash(rna_seq+struct)}.svg"
        RNA.svg_rna_plot(rna_seq, struct, str(path))
        time.sleep(0.1)
        svg_bytes = path.read_bytes()
        b64 = base64.b64encode(svg_bytes).decode('utf-8')
    except Exception as e:
        err = ('<svg xmlns="http://www.w3.org/2000/svg" width="400" height="200">'
               f'<text x="50" y="100" fill="red">Error: {e}</text></svg>')
        b64 = base64.b64encode(err.encode()).decode('utf-8')
    return f"data:image/svg+xml;base64,{b64}"


def fold(rna_seq: str, gt_struct: str):
    """展示 Ground Truth、ViennaRNA 与模型预测的结构对比"""
    if not rna_seq.strip():
        return "", "", "", ""
    # Ground Truth: 用户输入优先
    ground = gt_struct.strip() if gt_struct and gt_struct.strip() else ""
    gt_uri = generate_svg_datauri(rna_seq, ground) if ground else ""

    # ViennaRNA 预测
    vienna_struct, vienna_energy = RNA.fold(rna_seq)
    vienna_uri = generate_svg_datauri(rna_seq, vienna_struct)

    # 模型预测
    result = ssp_model.inference(rna_seq)
    pred = "".join(result.get('predictions', []))
    if ss_validity_loss(pred):
        for i in find_invalid_positions(pred):
            pred = pred[:i] + '.' + pred[i+1:]
    pred_uri = generate_svg_datauri(rna_seq, pred)

    # 统计信息
    match_gt = (sum(a==b for a,b in zip(ground, pred)) / len(ground)) if ground else 0
    match_vienna = sum(a==b for a,b in zip(vienna_struct, pred)) / len(vienna_struct)
    stats = (
        f"GT↔Pred Match: {match_gt:.2%}" + (" | " if ground else "") +
        f"Vienna↔Pred Match: {match_vienna:.2%}"
    )

    # 合并 HTML：三图水平排列
    combined = (
        '<div style="display:flex;justify-content:space-around;">'
        f'{f"<div><h4>Ground Truth</h4><img src=\"{gt_uri}\" style=\"max-width:100%;height:auto;\"/></div>" if ground else ""}'
        f'<div><h4>ViennaRNA</h4><img src=\"{vienna_uri}\" style=\"max-width:100%;height:auto;\"/></div>'
        f'<div><h4>Prediction</h4><img src=\"{pred_uri}\" style=\"max-width:100%;height:auto;\"/></div>'
        '</div>'
    )
    return ground, vienna_struct, pred, stats, combined


def sample_rna_sequence():
    """从测试集中抽样，返回序列与 Ground Truth 结构"""
    try:
        exs = [json.loads(l) for l in open('toy_datasets/Archive2/test.json')]
        ex = exs[np.random.randint(len(exs))]
        return ex['seq'], ex.get('label','')
    except Exception as e:
        return f"加载样本出错: {e}", ""

# Gradio UI
with gr.Blocks(css="""
.heading {text-align:center;color:#2a4365;}
.controls {display:flex;gap:10px;margin:20px 0;}
.status {padding:10px;background:#f0f4f8;border-radius:4px;white-space:pre;}
""") as demo:
    gr.Markdown("# RNA 结构预测对比", elem_classes="heading")
    with gr.Row():
        rna_input = gr.Textbox(label="RNA 序列", lines=3)
        structure_input = gr.Textbox(label="Ground Truth 结构 (可选)", lines=3)
    with gr.Row(elem_classes="controls"):
        sample_btn = gr.Button("抽取样本")
        run_btn = gr.Button("预测并对比", variant="primary")
    stats_out    = gr.Textbox(label="统计信息", interactive=False, elem_classes="status")
    gt_out       = gr.Textbox(label="Ground Truth", interactive=False)
    vienna_out   = gr.Textbox(label="ViennaRNA 结构", interactive=False)
    pred_out     = gr.Textbox(label="Prediction 结构", interactive=False)
    combined_view= gr.HTML(label="三图对比视图")

    run_btn.click(
        fold,
        inputs=[rna_input, structure_input],
        outputs=[gt_out, vienna_out, pred_out, stats_out, combined_view]
    )
    sample_btn.click(
        sample_rna_sequence,
        outputs=[rna_input, structure_input]
    )

    demo.launch(share=True)


ModuleNotFoundError: No module named 'gradio'

### Conclusion
In this demonstration, we have shown how to fine-tune a genomic foundation model for RNA secondary structure prediction using the OmniGenome package. We have also shown how to use the trained model for inference and how to create a web demo for RNA secondary structure prediction. We hope this demonstration will help you get started with genomic foundation model development using OmniGenome.